This Query is created by user sohamjcpp

In [2]:
%% sql
```
WITH Premia_Pcom_customer AS (
    SELECT 
        cust_code AS intermediary_code,
        cust_name AS intermediary_name,
        intermediary_type,
        agent_mobile_no,
        agent_email_id
    FROM Premia
    WHERE LEFT(cust_code, 2) = (SELECT dim_intermediary_type FROM policy_master_agg)
),
Bancs_T_RPDB_POL_PARTY AS (
    SELECT 
        cust_code AS intermediary_code,
        party_name AS intermediary_name
    FROM Bancs
    WHERE party_code IN (SELECT intermediary FROM policy_master)
),
Derived AS (
    SELECT
         CASE
            WHEN (CURRENT_DATE - aml_acknowledged_timestamp) >= 365 THEN 'Yes'
            ELSE 'No'
         END AS due_aml_training
    FROM vo_login
)
SELECT 
    intermediary_code,
    intermediary_name,
    intermediary_type,
    agent_mobile_no,
    agent_email_id,
    is_aml_acknowledged,
    aml_acknowledged_timestamp,
    due_aml_training,
    (CURRENT_DATE - 1) AS reported_date,
    CURRENT_TIMESTAMP AS load_timestamp,
    fufiller_code,
    fulfiller_name,
    office_code,
    office_name,
    zone_name,
    mkt_zone_name
FROM Premia_Pcom_customer
LEFT JOIN Bancs_T_RPDB_POL_PARTY ON Premia_Pcom_customer.intermediary_code = Bancs_T_RPDB_POL_PARTY.intermediary_code
LEFT JOIN Derived ON 1=1
LEFT JOIN agg_policy_master_report_daily ON Premia_Pcom_customer.intermediary_code = agg_policy_master_report_daily.intermediary_code
    AND agg_policy_master_report_daily.policy_from_date = (
        SELECT MAX(policy_from_date) 
        FROM agg_policy_master_report_daily 
        WHERE intermediary_code = Premia_Pcom_customer.intermediary_code
    )
```